# Using AI Platform Pipelines (Hosted Kubeflow Pipelines) from a notebook


[Cloud AI Platform Pipelines](https://cloud.google.com/ai-platform/pipelines/docs) , currently in Beta, provides a way to deploy robust, repeatable machine learning pipelines along with monitoring, auditing, version tracking, and reproducibility, and gives you an easy-to-install, secure execution environment for your ML workflows. AI Platform Pipelines is based on  [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/) (KFP) installed on a  [Google Kubernetes Engine (GKE)](https://cloud.google.com/kubernetes-engine)  cluster, and can run pipelines specified via both the KFP and TFX SDKs. See  [this blog post](https://cloud.google.com/blog/products/ai-machine-learning/introducing-cloud-ai-platform-pipelines) for more detail on the Pipelines tech stack. You can create an AI Platform Pipelines installation with just a few clicks. After installing, you access AI Platform Pipelines by visiting the AI Platform Panel in the  [Cloud Console](https://console.cloud.google.com/).


In this notebook:

- First we'll define a KFP pipeline based on predefined [_reusable components_](https://www.kubeflow.org/docs/pipelines/sdk/component-development/), and launch a pipeline run from the notebook. 
- Then we'll show how to define a new component that's based on a python function-- a so-called 'lightweight component'.
- The notebook will also show how to grab the ID of a pipeline previously uploaded to the dashboard and use that to kick off a run.
- Finally, we'll show how to trigger a pipeline run when new data becomes available, via Cloud Functions.

## Set up to run this example

### Create an AI Platform Notebooks instance

If you're not doing so already, run this notebook on an AI Platform Notebook instance. See setup instructions [here](https://cloud.google.com/ai-platform/notebooks/docs).  (It's possible to run the notebook using other Jupyter environments, but that requires some additional auth setup that we won't cover here).  Once your notebook instance is set up, you should be able to use [this link](https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=KFP%20from%20a%20notebook&download_url=https%3A%2F%2Fraw.githubusercontent.com%2Famygdala%2Fcode-snippets%2Fmaster%2Fml%2Fnotebook_examples%2Fcaipp%2Fkfp_in_a_notebook.ipynb&url=https%3A%2F%2Fgithub.com%2Famygdala%2Fcode-snippets%2Fblob%2Fmaster%2Fml%2Fnotebook_examples%2Fcaipp%2Fkfp_in_a_notebook.ipynb) to upload the notebook.

### Install AI Platform Pipelines

The example assumes a Pipeline Installation set up as described [here](https://github.com/amygdala/code-snippets/tree/master/ml/kubeflow-pipelines/keras_tuner) (however, for this notebook you won't need the preemptible node pool, so you can skip that part if you like).  The training step in the workflow does assume a GPU-enabled cluster node, though as indicated below, you can edit the pipeline definition to change that.

### Install the KFP SDK

Next, we'll install the KFP SDK, and then restart the kernel so it's available for import.

In [ ]:
!pip install --user -U kfp kfp-server-api

In [ ]:
# Restart kernel after the installs
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## Define and run a pipeline

Now we'll create a pipeline to train a model, then serve it using [TF-serving](xxx).  We'll be training a [Keras](xxx) model to predict duration of London bike rentals given info about the start and end station as well as day of week, current weather, and other info. See [this README](xxx) for more detail.

First we'll do some imports:

In [ ]:
import kfp  # the Pipelines SDK. 
from kfp import compiler
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.components as comp

We'll use KFP [reusable components](https://www.kubeflow.org/docs/pipelines/sdk/component-development/) to construct the pipeline:

In [ ]:

train_op = comp.load_component_from_url(
  'https://raw.githubusercontent.com/amygdala/code-snippets/master/ml/kubeflow-pipelines/keras_tuner/components/train_component.yaml'
  )
serve_op = comp.load_component_from_url(
  'https://raw.githubusercontent.com/amygdala/code-snippets/master/ml/kubeflow-pipelines/keras_tuner/components/serve_component.yaml'
  )

tb_op = comp.load_component_from_url(
  'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/tensorflow/tensorboard/prepare_tensorboard/component.yaml' 
  )

Now we'll define the pipeline itself, using the above component definitions. 

This pipeline first sets up a TensorBoard visualization for monitoring the training run. Then it starts the training. Once training is finished, it uses TF-serving to set up a service on the Pipelines GKE cluster.

If you don't want to run the training step on GPUs, you can comment out the `train.set_gpu_limit(2)` line below.


In [ ]:
@dsl.pipeline(
  name='bikes_weather',
  description='Model bike rental duration given weather'
)
def bikes_weather( 
  train_epochs: int = 5,
  working_dir: str = 'gs://YOUR/GCS/PATH',  # for the full training jobs
  data_dir: str = 'gs://aju-dev-demos-codelabs/bikes_weather/',
  steps_per_epoch: int = -1 ,  # if -1, don't override normal calcs based on dataset size
  hptune_params: str = '[{"num_hidden_layers": %s, "learning_rate": %s, "hidden_size": %s}]' % (3, 1e-2, 64)
  ):

  # create TensorBoard viz for the training run
  tb_viz = tb_op(
    log_dir_uri='%s/%s' % (working_dir, dsl.RUN_ID_PLACEHOLDER)
  )

  train = train_op(
    data_dir=data_dir,
    workdir='%s/%s' % (tb_viz.outputs['log_dir_uri'], 0),
    tb_dir=tb_viz.outputs['log_dir_uri'],
    epochs=train_epochs, steps_per_epoch=steps_per_epoch,
    hp_idx=0, 
    hptune_results=hptune_params
    )

  serve = serve_op(
    model_path=train.outputs['train_output_path'],
    model_name='bikesw',
    namespace='default'
    )
  train.set_gpu_limit(2)

You can see that data is being passed between the pipeline ops. [Here's a tutorial](https://gist.github.com/amygdala/bfa0f599a4814b3261367f558a852bfe) that goes into how that works in more detail.

Next we'll compile the pipeline (creating a local archive file):

In [ ]:
compiler.Compiler().compile(bikes_weather, 'bikes_weather.tar.gz')

Then create a client object, and using that client, create (or get) an _Experiment_ (which lets you create semantic groupings of pipeline runs).

You'll need to set the correct host endpoint for your pipelines installation when you create the client.  Visit the [Pipelines panel in the Cloud Console](https://console.cloud.google.com/ai-platform/pipelines/clusters) and click on the **SETTINGS** gear for the desired installation to get its endpoint.

In [ ]:
# CHANGE THIS with the info for your KFP cluster installation
client = kfp.Client(host='xxxxxxxx-dot-us-centralx.pipelines.googleusercontent.com')

In [ ]:
exp = client.create_experiment(name='bw_expers')  # this is a 'get or create' call

Next, set some pipeline params, and run the pipeline, passing it the path to the compiled file:

In [ ]:
WORKING_DIR = 'gs://YOUR_GCS/PATH'
TRAIN_EPOCHS = 2

In [ ]:
run = client.run_pipeline(exp.id, 'bw_test', 'bikes_weather.tar.gz',
                          params={'working_dir': WORKING_DIR, 'train_epochs': TRAIN_EPOCHS})

Once you've kicked off the run, click the generated link to see the pipeline run in the Kubeflow Pipelines dashboard of your pipelines installation. (See the last section of this notebook for more info on how to use your trained and deployed model for prediction).

**Note**: It's also possible to start a pipeline run directly from the pipeline function definition, skipping the local compilation, like this:
```python
kfp.Client(host=kfp_endpoint).create_run_from_pipeline_func(<pipeline_function_name>, arguments={})
```

One thing that might have occurred to you with the example thus far: what if the trained model's accuracy is not that great, and we don't necessarily want to deploy it?  We'll address that in the next section.

## Defining a new 'lightweight component' based on a python function

'Lightweight' KFP python components allow you to create a component from a python function definition, and do not require you to build a new container image for every code change. They're helpful for fast iteration in a notebook environment. You can read more [here](https://github.com/kubeflow/pipelines/blob/master/samples/core/lightweight_component/lightweight_component.ipynb).

In this section, we'll create a lightweight component that uses training metrics info to decide whether to deploy a model.
We'll pass a "threshold" dict as a component arg, and compare those thresholds to the metrics values, and use that info to decide whether or not to deploy.  Then we'll output a string indicating the decision. 

(This is a pretty simple approach, that we're using for illustrative purposes; for production models you'd probably want to do more sophisticated analyses. The [TFMA library](https://www.tensorflow.org/tfx/model_analysis/get_started) might be of interest).

Then we'll update the pipeline to use the new component. In the new pipeline definition below, we'll make the 'serve' step conditional on that output.

First, we'll define the component function, `eval_metrics`:

In [ ]:
from typing import NamedTuple

def eval_metrics(
  metrics: str,
  thresholds: str
) -> NamedTuple('Outputs', [('deploy', str)]):

  import json
  import logging

  def regression_threshold_check(metrics_info):
    # ...
    for k, v in thresholds_dict.items():
      logging.info('k {}, v {}'.format(k, v))
      if k in ['root_mean_squared_error', 'mae']:
        if metrics_info[k][-1] > v:
          logging.info('{} > {}; returning False'.format(metrics_info[k][0], v))
          return ('False', )
    return ('deploy', )

  logging.getLogger().setLevel(logging.INFO)

  thresholds_dict = json.loads(thresholds)
  logging.info('thresholds dict: {}'.format(thresholds_dict))
  logging.info('metrics: %s', metrics)
  metrics_dict = json.loads(metrics)

  logging.info("got metrics info: %s", metrics_dict)
  res = regression_threshold_check(metrics_dict)
  logging.info('deploy decision: %s', res)
  return res


Next we'll create a 'container op' from that function definition, via the `funct_to_container_op` method. As one of the method args, we specify the base container image that will run the function. 
Here, we're using one of the [Deep Learning Container images](https://cloud.google.com/ai-platform/deep-learning-containers/docs/).  (Admittedly, this is overkill for this simple function).

In [ ]:
eval_metrics_op = comp.func_to_container_op(eval_metrics, base_image='gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest')

Now, we can define a new pipeline that uses the new op and makes the model serving conditional.

The new `eval_metrics_op` takes as an input one of the `train_op` outputs, which outputs a metrics dict. (We "cheated" a bit, as the training component was already designed to output this info; in other cases you might end up defining a new version of such an op that outputs the new info you need).

Then, we'll wrap the serving op in a conditional; we won't set up a TF-serving service unless the `eval_metrics` op has certified that it is okay.

Note that this new version of the pipeline also has a new input parameter— the `thresholds` dict.

In [ ]:
@dsl.pipeline(
  name='bikes_weather_metrics',
  description='Model bike rental duration given weather'
)
def bikes_weather_metrics( 
  train_epochs: int = 2,
  working_dir: str = 'gs://YOUR/GCS/PATH',  # for the full training jobs
  data_dir: str = 'gs://aju-dev-demos-codelabs/bikes_weather/',
  steps_per_epoch: int = -1 ,  # if -1, don't override normal calcs based on dataset size
  hptune_params: str = '[{"num_hidden_layers": %s, "learning_rate": %s, "hidden_size": %s}]' % (3, 1e-2, 64),
  thresholds: str = '{"root_mean_squared_error": 2000}'
  ):

  # create TensorBoard viz for the parent directory of all training runs, so that we can
  # compare them.
  tb_viz = tb_op(
    log_dir_uri='%s/%s' % (working_dir, dsl.RUN_ID_PLACEHOLDER)
  )

  train = train_op(
    data_dir=data_dir,
    workdir='%s/%s' % (tb_viz.outputs['log_dir_uri'], 0),
    tb_dir=tb_viz.outputs['log_dir_uri'],
    epochs=train_epochs, steps_per_epoch=steps_per_epoch,
    hp_idx=0, 
    hptune_results=hptune_params
    )

  eval_metrics = eval_metrics_op(
    thresholds=thresholds,
    metrics=train.outputs['metrics_output_path'],
    )

  with dsl.Condition(eval_metrics.outputs['deploy'] == 'deploy'):
    serve = serve_op(
      model_path=train.outputs['train_output_path'],
      model_name='bikesw',
      namespace='default'
      )
  train.set_gpu_limit(2)

Now, as before, we can compile and then run the pipeline.

In [ ]:
compiler.Compiler().compile(bikes_weather_metrics, 'bikes_weather_metrics.tar.gz')

In [ ]:
run2 = client.run_pipeline(exp.id, 'bw_metrics_test', 'bikes_weather_metrics.tar.gz',
                          params={'working_dir': WORKING_DIR, 'train_epochs': TRAIN_EPOCHS
                                 # 'thresholds': THRESHOLDS
                                 })

Again, once you've kicked off the run, click the link that will appear above to view the run in the Kubeflow Pipelines dashboard.

## Starting a pipeline run given the pipeline's ID

In the sections above, we compiled a pipeline definition to a local archive files, then passed the path to the file as part of the `run_pipeline` method.

It's also possible to start a pipeline run given the ID (not display name) of an already-uploaded pipeline. You can find these IDs via the Kubeflow Pipeline dashboard, but it's also possible to grab them programmatically.

As an example, we'll define a utility function to grab a pipeline's ID given its name, then use the given ID in the `run_pipeline` call.

> Note: Rather confusingly, with the the `run_pipeline` calls above, the created pipelines aren't actually given a display name (though they still have an ID). So the `get_pipeline_id` utility defined below is designed to be used for pipelines explicitly uploaded to the dashboard from the "Pipelines" tag, _not_ pipelines defined as we've done above in this notebook. You can visit the dashboard to get the IDs of the pipelines you've just created, or find them in the list of all pipelines.

In [ ]:
# you can get a list of all the pipelines like this...
# pipelines_list = client.list_pipelines()
# pipelines_list

In [ ]:
import json
_FILTER_OPERATIONS = {"UNKNOWN": 0,
    "EQUALS" : 1,
    "NOT_EQUALS" : 2,
    "GREATER_THAN": 3,
    "GREATER_THAN_EQUALS": 5,
    "LESS_THAN": 6,
    "LESS_THAN_EQUALS": 7}

def get_pipeline_id(client, name):
  """Find the id of a pipeline by name.
  Args:
    name: Pipeline name.
  Returns:
    Returns the pipeline id if a pipeline with the name exists.
  """
  pipeline_filter = json.dumps({
    "predicates": [
      {
        "op":  _FILTER_OPERATIONS["EQUALS"],
        "key": "name",
        "stringValue": name,
      }
    ]
  })
  result = client._pipelines_api.list_pipelines(filter=pipeline_filter)
  if result.pipelines is None:
    return None
  if len(result.pipelines)==1:
    return result.pipelines[0].id
  elif len(result.pipelines)>1:
    raise ValueError("Multiple pipelines with the name: {} found, the name needs to be unique".format(name))
  return None

The following function call assumes the existence of a named pipeline that you've uploaded to the Kubeflow Pipelines dashboard:

In [ ]:
# Replace with the name of your pipeline. 
get_pipeline_id(client, 'YOUR_PIPELINE_NAME')

**Edit the following cell** to use the output pipeline ID. (And modify the params dict if you're using a different pipeline).

In [ ]:
run3 = client.run_pipeline(exp.id, 'bw_metrics_test2', pipeline_id='YOUR_PIPELINE_ID',
                          params={'working_dir': WORKING_DIR, 'train_epochs': TRAIN_EPOCHS
                                 })

## Triggering a pipeline run when new data comes in

Often, you may want to trigger a new run of the training pipeline when new data becomes available.

We can use [Cloud Functions](https://cloud.google.com/functions/docs/) (GCF) to do this.  This section walks through how you could set this up. 

> Note: Depending upon ML workflow context, in actuality you might want to fine-tune an existing model, or retrain on a different window of data than you did originally.  For this example we're keeping things simple and just triggering another full model training job on the dataset after new data is added.

In this notebook, **we're skipping some of the prereqs, including some required auth setup.  See [this notebook](https://github.com/amygdala/code-snippets/blob/master/ml/notebook_examples/functions/hosted_kfp_gcf.ipynb) for the details.**

The first step is to identify a GCS 'trigger bucket'.  **It should be a new bucket or one that you don't use for anything else**. We'll set things up so that if any objects are added to this bucket, or modified, the GCF function will run, and it in turn will deploy a run of the pipeline.  Edit and run the following to set your bucket:

In [ ]:
%env TRIGGER_BUCKET=YOUR_TRIGGER_BUCKET

Next, copy the input directory used with the example model to your own trigger bucket, which you can do as follows:

In [ ]:
%%bash
gsutil cp 'gs://aju-dev-demos-codelabs/bikes_weather/*' gs://${TRIGGER_BUCKET}/bikes_weather_temp
gsutil ls gs://${TRIGGER_BUCKET}/bikes_weather_temp

Then, delete one of the files from your temp data directory (we'll shortly add it back to trigger the GCF function):

In [ ]:
%%bash
gsutil rm gs://${TRIGGER_BUCKET}/bikes_weather_temp/train-bw000000000007.csv
gsutil ls gs://${TRIGGER_BUCKET}/bikes_weather_temp    

Next, we'll create a subdirectory for the GCF definition:

In [ ]:
%%bash
mkdir -p functions

We'll first create a requirements.txt file, to indicate what packages the GCF code requires to be installed.

In [ ]:
%%writefile functions/requirements.txt
kfp

Next we'll define the GCF function.

**Before you run the next cell, edit** the `HOST`, `WORKING_DIR`, `DATA_DIR`, and `PIPELINE_ID` values. Set `DATA_DIR` to the path under which your copied input files live, e.g. `gs://${TRIGGER_BUCKET}/bikes_weather_temp/`.  Include the trailing slash. Set `PIPELINE_ID` to the ID of the pipeline that you want to run.  Modify the params dict as necessary for the pipeline you're running.

In [ ]:
%%writefile functions/main.py
import logging
import datetime
import logging
import time
 
import kfp
import kfp.compiler as compiler
import kfp.dsl as dsl
 
import requests
 
# TODO: replace with your Pipelines endpoint URL
HOST = 'xxxxxxxx-dot-us-centralx.pipelines.googleusercontent.com'
# TODO: replace with your working dir
WORKING_DIR = 'gs://YOUR_WORKING_DIR'
TRAIN_EPOCHS = 2
DATA_DIR = 'gs://PATH_TO/YOUR_COPIED_INPUT_DATA/'  # include the trailing slash
PIPELINE_ID = 'YOUR_PIPELINE_ID'

 
def get_access_token():
  url = 'http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/token'
  r = requests.get(url, headers={'Metadata-Flavor': 'Google'})
  r.raise_for_status()
  access_token = r.json()['access_token']
  return access_token
 
def hosted_kfp_test(data, context):
  logging.info('Event ID: {}'.format(context.event_id))
  logging.info('Event type: {}'.format(context.event_type))
  logging.info('Data: {}'.format(data))
  logging.info('Bucket: {}'.format(data['bucket']))
  logging.info('File: {}'.format(data['name']))
  file_uri = 'gs://%s/%s' % (data['bucket'], data['name'])
  logging.info('Using file uri: %s', file_uri)
  
  logging.info('Metageneration: {}'.format(data['metageneration']))
  logging.info('Created: {}'.format(data['timeCreated']))
  logging.info('Updated: {}'.format(data['updated']))
  
  token = get_access_token() 
  logging.info('attempting to launch pipeline run.')
  ts = int(datetime.datetime.utcnow().timestamp() * 100000)
  client = kfp.Client(host=HOST, existing_token=token)
  exp = client.create_experiment(name='gcstriggered')  # this is a 'get or create' op
  res = client.run_pipeline(exp.id, 'bwmetrics_' + str(ts), 
      pipeline_id=PIPELINE_ID,
      params={'working_dir': WORKING_DIR, 'train_epochs': TRAIN_EPOCHS,
              'data_dir': DATA_DIR}
  )
  logging.info(res)


Deploy the GCF function as follows. (You'll need to wait a moment or two for output of the deployment to display in the notebook). You can also run this command from a notebook terminal window in the functions subdirectory.

In [ ]:
%%bash
cd functions
gcloud functions deploy gcs_test --runtime python37 --trigger-resource ${TRIGGER_BUCKET} --trigger-event google.storage.object.finalize

Once the Cloud Function is successfully deployed, trigger the pipeline run by adding a new file to the `DATA_DIR`, e.g.:

In [ ]:
%%bash
gsutil cp gs://aju-dev-demos-codelabs/bikes_weather/train-bw000000000007.csv gs://${TRIGGER_BUCKET}/bikes_weather_temp/train-bw000000000007.csv

You should see the newly launched pipeline running in the Kubeflow Pipelines dashboard.

> **Important note**: For simplicity, this example is a bit unrealistic in that for each file added to or updated in the bucket, a new separate Pipeline run is launched.  So it would not suit for cases where multiple new files are added to a directory at once. A more general solution might keep the data directory separate from the trigger bucket, monitor it for new files, and periodically write a 'new file' notification to the trigger bucket.  
(So, once you have your Cloud Function trigger set up, be careful of copying multiple new files to the trigger bucket at once).

## More detail on the code, and requesting predictions from your model

This notebook didn't focus on the details of the pipeline component (step) implementations.  The training component uses a Keras model (TF 2.3). The serving component uses [TF-serving](https://www.tensorflow.org/tfx/guide/serving): once the serving service is up and running, you can send prediction requests to your trained model.

You can find more detail on these components, and an example of sending a prediction request, [here](https://github.com/amygdala/code-snippets/tree/master/ml/kubeflow-pipelines/keras_tuner).


----------------------------
Copyright 2020, Google, LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.